### 1. Load Packages

In [1]:
import openai
import pandas as pd
import re

### 2. Load Tokens

In [2]:
openai.api_key = "To be filled"

### 3. Load Data or Text

In [3]:
df1 = pd.read_csv('processed_60900_61899.csv')
df2 = pd.read_csv('processed_177000_177999.csv')

In [4]:
df1

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,60900,60900,6555,we already have congestion pricing for taxis &...,2021-07-09 00:02:34+00:00,2021-07-09,2021-27
1,60901,60901,6556,an the best cannot be that much i am sorry htt...,2021-07-08 23:44:03+00:00,2021-07-08,2021-27
2,60902,60902,6557,handicap uber's is getting out of hand,2021-07-08 23:40:10+00:00,2021-07-08,2021-27
3,60903,60903,6558,bitchh i would calmly get in a cab laugh ill b...,2021-07-08 23:16:21+00:00,2021-07-08,2021-27
4,60904,60904,6559,take a cab. you are getting hepatitis walking ...,2021-07-08 23:15:04+00:00,2021-07-08,2021-27
...,...,...,...,...,...,...,...
995,61895,61895,7550,so...i confess ...i walked to target &amp; bou...,2021-06-17 23:27:09+00:00,2021-06-17,2021-24
996,61896,61896,7551,"uber, ferry, plane, train today",2021-06-17 23:00:01+00:00,2021-06-17,2021-24
997,61897,61897,7552,"hi, i order some food about a hour ago and i h...",2021-06-17 22:08:29+00:00,2021-06-17,2021-24
998,61898,61898,7553,i was there. it was an uber. none of the other...,2021-06-17 21:54:38+00:00,2021-06-17,2021-24


In [5]:
df2

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,177000,177000,7102,please kick japan out of my country they are s...,2021-11-26 13:17:13+00:00,2021-11-26,2021-47
1,177001,177001,7103,once i learned about correctional exercises an...,2021-11-26 12:41:27+00:00,2021-11-26,2021-47
2,177002,177002,7104,empty train cars &gt;&gt;,2021-11-26 12:22:52+00:00,2021-11-26,2021-47
3,177003,177003,7105,that emerald parking lot is about 1 mile walk ...,2021-11-26 12:14:05+00:00,2021-11-26,2021-47
4,177004,177004,7106,"that is going to take time, i need car by end ...",2021-11-26 11:48:54+00:00,2021-11-26,2021-47
...,...,...,...,...,...,...,...
995,177995,177995,8097,how about getting all trash off the sidewalk? ...,2021-06-03 02:03:54+00:00,2021-06-03,2021-22
996,177996,177996,8098,get trash off the sidewalk. trash corals in th...,2021-06-03 02:00:17+00:00,2021-06-03,2021-22
997,177997,177997,8099,people of philadelphia find trae young's hotel...,2021-06-03 01:55:00+00:00,2021-06-03,2021-22
998,177998,177998,8100,ring rain coming back for more and do not you ...,2021-06-03 01:40:50+00:00,2021-06-03,2021-22


### 4.Chain of thought

In [6]:
def generate_chain_of_thought(tweet):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You're an assistant skilled in understanding public transport experiences from tweets. Provide an analysis based on the tweet."
            },
            {
                "role": "user",
                "content": f"Given the tweet: '{tweet}', carefully analyze the content and context to determine the travel experience shared by the tweet's author. Considering that multiple modes of transportation might be mentioned, focus on identifying the primary mode the author is describing as their own experience. Follow these steps in your analysis:\n\n"
                           "1. **Identify the Primary Mode of Transportation:** Examine the tweet for mentions of travel modes (car, subway, bus, bike, taxi). Determine which mode the author specifically refers to as their own means of transportation, if directly mentioned or implied through context.\n"
                           "2. **Assess Sentiment:** Once the primary mode is identified, evaluate the sentiment the author expresses towards this mode of transportation—positive, negative, or neutral.\n"
                           "3. **Extract Reasons:** Delve into the reasons behind the author's sentiment towards the primary travel mode. Identify any specific experiences, aspects, or observations mentioned that influence their view.\n\n"
                           "Conclude your analysis with a structured summary of your findings in the following format: "
                           "{travel_mode: '<mode>', sentiment: '<sentiment>', reasons: '<reasons>'}."
            }
        ],
        temperature=0.5,
        max_tokens=250
    )
    # Ensure the response's 'content' is extracted correctly
    return response.choices[0].message.content if response.choices else "No response"

### 5. parse answer

In [7]:
def parse_response(response_text):
    # Initialize default values
    travel_mode = 'NA'
    sentiment = 'neutral'
    reasons = 'No specific reasons given.'

    # Use regular expressions to extract the data
    try:
        travel_mode_match = re.search(r"travel_mode:\s*'([^']*)'", response_text)
        sentiment_match = re.search(r"sentiment:\s*'([^']*)'", response_text)
        reasons_match = re.search(r"reasons:\s*'([^']*)'", response_text)

        if travel_mode_match:
            travel_mode = travel_mode_match.group(1)
        if sentiment_match:
            sentiment = sentiment_match.group(1)
        if reasons_match:
            reasons = reasons_match.group(1)
    except Exception as e:
        print(f"Error parsing response: {e}")
        # Fallback to default values if parsing fails

    return travel_mode, sentiment, reasons

### 5.Outcome

**(1) df1: processed_60900_61899.csv**

In [8]:
# test
tweet = df1.iloc[0]['processed_txt']
response_text = generate_chain_of_thought(tweet)
travel_mode, sentiment, reasons = parse_response(response_text)
travel_mode, sentiment, reasons

('NA', 'neutral', 'No specific reasons given.')

In [9]:
df1_res = pd.DataFrame(columns=['Tweet', 'Travel Mode', 'Sentiment', 'Reasons'])

# Iterate through the original DataFrame
for index, row in df1.iterrows():
    tweet = row['processed_txt']  # Assuming 'processed_txt' contains the tweet text
    response_text = generate_chain_of_thought(tweet)
    travel_mode, sentiment, reasons = parse_response(response_text)
    
    # Append the results to the new DataFrame
    df1_res = df1_res.append({'Tweet': tweet, 'Travel Mode': travel_mode, 'Sentiment': sentiment, 'Reasons': reasons}, ignore_index=True)

# Save the new DataFrame to a new CSV file
df1_res.to_csv('result_df1.csv', index=False)

In [10]:
df1_res

,Tweet,Travel Mode,Sentiment,Reasons
0,we already have congestion pricing for taxis &...,car,negative,The author expresses a negative sentiment towa...
1,an the best cannot be that much i am sorry htt...,Public Transportation (Potentially Bus or Subway),neutral,No specific reasons given.
2,handicap uber's is getting out of hand,Uber,negative,Possible reasons for the negative sentiment in...
3,bitchh i would calmly get in a cab laugh ill b...,taxi,negative,The author expresses a negative sentiment towa...
4,take a cab. you are getting hepatitis walking ...,cab/taxi,negative,The author expresses a negative sentiment towa...
...,...,...,...,...
995,so...i confess ...i walked to target &amp; bou...,Uber,positive,The author found Uber convenient and necessary...
996,"uber, ferry, plane, train today",train,neutral,The author
997,"hi, i order some food about a hour ago and i h...",Food delivery service,Negative,The author expresses frustration about not hav...
998,i was there. it was an uber. none of the other...,Uber,Surprised/Disbelief,The author


**(2) df2: processed_177000_177999.csv**

In [11]:
df2_res = pd.DataFrame(columns=['Tweet', 'Travel Mode', 'Sentiment', 'Reasons'])

# Iterate through the original DataFrame
for index, row in df2.iterrows():
    tweet = row['processed_txt']  # Assuming 'processed_txt' contains the tweet text
    response_text = generate_chain_of_thought(tweet)
    travel_mode, sentiment, reasons = parse_response(response_text)
    
    # Append the results to the new DataFrame
    df2_res = df2_res.append({'Tweet': tweet, 'Travel Mode': travel_mode, 'Sentiment': sentiment, 'Reasons': reasons}, ignore_index=True)

# Save the new DataFrame to a new CSV file
df2_res.to_csv('result_df2.csv', index=False)

In [12]:
df2_res

,Tweet,Travel Mode,Sentiment,Reasons
0,please kick japan out of my country they are s...,car,negative,The author expresses negative sentiment toward...
1,once i learned about correctional exercises an...,Not Applicable,Not Applicable,The tweet focuses more on the author
2,empty train cars &gt;&gt;,train,positive,The author enjoys the comfort and peace of hav...
3,that emerald parking lot is about 1 mile walk ...,shuttle bus,negative,The author expresses a negative sentiment towa...
4,"that is going to take time, i need car by end ...",car,neutral,The author expresses a practical need for a ca...
...,...,...,...,...
995,how about getting all trash off the sidewalk? ...,car,negative,Frustration with trash on the sidewalk and pri...
996,get trash off the sidewalk. trash corals in th...,car,negative,The author is frustrated with the lack of spac...
997,people of philadelphia find trae young's hotel...,car,negative,The author expresses frustration and annoyance...
998,ring rain coming back for more and do not you ...,car,positive,The author expresses a positive sentiment towa...


## 6.  Output Evaluation

In [5]:
df1_res = pd.read_csv('result_df1.csv')
df1

,Tweet,Travel Mode,Sentiment,Reasons
0,we already have congestion pricing for taxis &...,car,negative,The author expresses a negative sentiment towa...
1,an the best cannot be that much i am sorry htt...,Public Transportation (Potentially Bus or Subway),neutral,No specific reasons given.
2,handicap uber's is getting out of hand,Uber,negative,Possible reasons for the negative sentiment in...
3,bitchh i would calmly get in a cab laugh ill b...,taxi,negative,The author expresses a negative sentiment towa...
4,take a cab. you are getting hepatitis walking ...,cab/taxi,negative,The author expresses a negative sentiment towa...
...,...,...,...,...
995,so...i confess ...i walked to target &amp; bou...,Uber,positive,The author found Uber convenient and necessary...
996,"uber, ferry, plane, train today",train,neutral,The author
997,"hi, i order some food about a hour ago and i h...",Food delivery service,Negative,The author expresses frustration about not hav...
998,i was there. it was an uber. none of the other...,Uber,Surprised/Disbelief,The author


In [15]:
def analyze_data(df):
    # Analyze 'Travel Mode'
    unique_travel_modes = df['Travel Mode'].unique()
    print("Distinct outcomes for Travel Mode:", unique_travel_modes)

    number_of_distinct_travel_modes = df['Travel Mode'].nunique()
    print("Number of distinct outcomes for Travel Mode:", number_of_distinct_travel_modes)

    travel_mode_counts = df['Travel Mode'].value_counts()
    print("Counts for each distinct outcome in Travel Mode:", travel_mode_counts)

    # Analyze 'Sentiment'
    unique_sentiments = df['Sentiment'].unique()
    print("Distinct outcomes for Sentiment:", unique_sentiments)

    number_of_distinct_sentiments = df['Sentiment'].nunique()
    print("Number of distinct outcomes for Sentiment:", number_of_distinct_sentiments)

    sentiment_counts = df['Sentiment'].value_counts()
    print("Counts for each distinct outcome in Sentiment:", sentiment_counts)

    # Calculate various rates
    extraction_rate = len(df[~df['Travel Mode'].isin(['na', 'NA','N/A'])]) / len(df)
    satisfaction_rate = len(df[df['Sentiment'] == 'positive']) / len(df)
    
    return {
        'Extraction Rate': extraction_rate,
        'Satisfaction Rate': satisfaction_rate,
    }


In [16]:
df1_res = pd.read_csv('result_df1.csv')
df1_res

,Tweet,Travel Mode,Sentiment,Reasons
0,we already have congestion pricing for taxis &...,car,negative,The author expresses a negative sentiment towa...
1,an the best cannot be that much i am sorry htt...,Public Transportation (Potentially Bus or Subway),neutral,No specific reasons given.
2,handicap uber's is getting out of hand,Uber,negative,Possible reasons for the negative sentiment in...
3,bitchh i would calmly get in a cab laugh ill b...,taxi,negative,The author expresses a negative sentiment towa...
4,take a cab. you are getting hepatitis walking ...,cab/taxi,negative,The author expresses a negative sentiment towa...
...,...,...,...,...
995,so...i confess ...i walked to target &amp; bou...,Uber,positive,The author found Uber convenient and necessary...
996,"uber, ferry, plane, train today",train,neutral,The author
997,"hi, i order some food about a hour ago and i h...",Food delivery service,Negative,The author expresses frustration about not hav...
998,i was there. it was an uber. none of the other...,Uber,Surprised/Disbelief,The author


In [17]:
analyze_data(df1_res)

Distinct outcomes for Travel Mode: ['car' 'Public Transportation (Potentially Bus or Subway)' 'Uber' 'taxi'
 'cab/taxi' 'Electric Vehicle/Car' 'cab' 'Lyft car service'
 'Subway (Washington D.C. Metro)' 'Water-based transportation'
 'Subway/Bus' 'bus' 'cabs and ubers' 'taxi/rideshare service' 'Lyft'
 'yellow cab' 'Ride-sharing service (e.g., Uber, Lyft)' 'bike' 'Taxi'
 'cycling' 'delivery service' 'Air travel (AirIndia)' 'Not applicable'
 'N/A (related to anime series)' 'Taxi/Ride-sharing service'
 'New York City public transportation' 'flight'
 'Public Transport (implied)' 'Not Applicable' nan 'Lyft (car)'
 'Various modes while drunk' 'Walking/Biking'
 'Food delivery/takeout service' 'food delivery service'
 'cabs or taxi services' 'taxi cabs' 'preferred mode of transportation'
 'train (Metro North)' 'bus/train' 'driver' 'food delivery'
 'betting on future bets for sports events' 'taxis' 'unspecified'
 'airplane' 'Car' 'Not specified'
 'anvil (potentially referring to a transportation 

{'Extraction Rate': 1.0, 'Satisfaction Rate': 0.147}